# Calculate OpenKnotScore

This notebook takes an input dataframe of sequences with reactivity data and various folding algorithm predictions and generates OpenKnotScores for the sequence library.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
from tqdm import tqdm
from openknotscore import scoring

# Trick to display dataframes with scrollbars
from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))

## Load data, Initialize constants

In [ ]:
# Load data
data = pd.read_pickle("../data/data+predictions.pkl")

# Remove any sequences without reactivity data
data = data[data['reactivity'].notna()]

# Grab the data frame column names that hold predictions for each model
prediction_tags = list(filter(lambda label: label.endswith("_PRED"), data.columns))

# Whether to filter out singlet base pairs (stacks/helices which only contain one pair)
FILTER_SINGLETS = False

## Calculate Eterna Classic Score

In [ ]:
# The Eterna Classic score measures how well a structure prediction matches the reactivity data
# We calculate the ECS for every model of every sequence in the library
def getECSperRow(row):
    # Apply the scoring function to each model prediction in the row
    df = row[prediction_tags].apply(
        scoring.calculateEternaClassicScore,
        args=(row['reactivity'], row['score_start_idx'] - 1, row['score_end_idx'] - 1, filter_singlets=FILTER_SINGLETS)
    )
    df = df.add_suffix('_ECS')
    return df

# Initialize a progress bar; important when working with large sequence libraries
tqdm.pandas(desc="Calculating Eterna Classic Score")

# Apply the scoring function to each row in the data frame
ecs = data.progress_apply(getECSperRow,axis=1,result_type='expand')

# Add the ECS score columns to the data 
output = pd.merge(data,ecs,how="left",left_index=True,right_index=True)

display(output)

## Calculate Crossed Pair Quality Scores

In [ ]:
# The CPQ scores are a pair of metrics that calculate how well the reactivity data
# supports the presence of crossed pairs in a predicted structure. The crossed pair
# score is measured against the entire structure, while the crossed pair quality score
# is measured against only the base pairs that are predicted to be in a crossed pair.
def getCPQperRow(row):
    # Apply the scoring function to each model prediction in the row
    df = row[prediction_tags].apply(
        scoring.calculateCrossedPairQualityScore,
        args=(row['reactivity'], row['score_start_idx'] - 1, row['score_end_idx'] - 1, filter_singlets=FILTER_SINGLETS)                   
    )
    df = df.add_suffix('_CPQ')
    return df

# Initialize a progress bar; important when working with large sequence libraries
tqdm.pandas(desc="Calculating Crossed Pair Quality Score")

# Apply the scoring function to each row in the data frame
cpq = data.progress_apply(getCPQperRow,axis=1,result_type='expand')

# Add the CPQ score columns to the data 
output = pd.merge(output,cpq,how="left",left_index=True,right_index=True)

display(output)

In [ ]:
# The OpenKnotScore is a metric that estimates how likely a sequence is to contain 
# a pseudoknot structure. The score is derived by averaging the Eterna Classic Score
# (measure of structure match to reactivity data) and the Crossed Pair Quality Score
# (measure of reactivity support for crossed pairs) across an ensemble of several
# structure predictions from various predictive models.

# Initialize a progress bar; important when working with large sequence libraries
tqdm.pandas(desc="Calculating OpenKnotScore")

# Apply the scoring function to each row in the data frame
oks = output.progress_apply(scoring.calculateOpenKnotScore,axis=1, args=(prediction_tags,)  )

# Add the OKS score columns to the data
complete = pd.merge(output,oks,how="left",left_index=True,right_index=True)

display(complete)

## Save output data

In [ ]:
complete.to_pickle('../data/data_processed.pkl')
# complete.to_csv('../data/data_processed.csv')